In [ ]:
def determineBucket(hand, left_shoulder, right_shoulder):
    if hand.y < left_shoulder.y:

        if hand.x < right_shoulder.x:
            return 0

        if hand.x < left_shoulder.x:
            return 1

        return 2

    if hand.x < right_shoulder.x:
        return 3

    if hand.x < left_shoulder.x:
        return 4

    return 5

In [ ]:


def trackBuckets(clip):
    left_buckets = [0] * 6
    right_buckets = [0] * 6

    for frame in clip.frames:
        # shoulders act as boundary markers for buckets
        left_shoulder = frame.pose_landmarks[11]
        right_shoulder = frame.pose_landmarks[12]
        left_shoulder.y = (left_shoulder.y + right_shoulder.y) / 2
        right_shoulder.y = left_shoulder.y

        left_wrist = frame.pose_landmarks[15]
        right_wrist = frame.pose_landmarks[16]

        if left_wrist.visibility > 0.9:
            left_idx = determineBucket(left_wrist, left_shoulder, right_shoulder)
            left_buckets[left_idx] += 1
        else:
            left_buckets[5] += 1

        if right_wrist.visibility > 0.9:
            right_idx = determineBucket(right_wrist, left_shoulder, right_shoulder)
            right_buckets[right_idx] += 1
        else:
            right_buckets[3] += 1

    return left_buckets, right_buckets

In [ ]:
import math


def cosine(vec1, vec2):
    vec_len = len(vec1)
    sum_of_prod = 0
    sum_of_square_1 = 0
    sum_of_square_2 = 0
    for i in range(vec_len):
        sum_of_prod += vec1[i] * vec2[i]
        sum_of_square_1 += vec1[i] * vec1[i]
        sum_of_square_2 += vec2[i] * vec2[i]

    return sum_of_prod / (math.sqrt(sum_of_square_1) * (math.sqrt(sum_of_square_2)))

In [ ]:
import pickle

data = []

with open("dataset/gestures/0.pkl", "rb") as reader:
    data.append(pickle.load(reader))

with open("dataset/gestures/6.pkl", "rb") as reader:
    data.append(pickle.load(reader))

buckets_a = trackBuckets(data[0].clips[0])
buckets_b = trackBuckets(data[0].clips[1])
buckets_c = trackBuckets(data[1].clips[0])

print(buckets_a)
print(buckets_b)
print(buckets_c)

print(cosine(buckets_a[0], buckets_b[0]))
print(cosine(buckets_a[1], buckets_b[1]))
print(cosine(buckets_a[0], buckets_c[0]))
print(cosine(buckets_a[1], buckets_c[1]))